### Load packages

In [1]:
import msprime
import numpy as np
import pandas as pd
import math
from random import shuffle

In [6]:
import plotly
import plotly.plotly as py
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
# plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username='jed.e.carlson', api_key='tv59AK36bWkTieq6qJdH')

### Set up simulation under Fu et al., 2013 out-of-Africa model with recent rapid growth

In [7]:
def out_of_africa(mu=1.5e-8, phi=0, length=1e4, n_afr=0, n_eur=0, debug=False):
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    
    # 220kya:
    # African population constant with Ne~7300
    N_A = 7310
    
    # 148kya:
    # instantaneous growth to Ne~14000
    T_AF = 148e3 / generation_time
    N_AF = 14474
    
    # 51kya:
    # non-AFR pops migrate OOA; bottlenecks to Ne~1800
    # migration between AFR occurs
    N_B = 1861
    T_SPLIT = 51e3 / generation_time
    m_AF_B = 15e-5
    
    # 23kya:
    # 2nd EUR bottlenecks to Ne~1000 & starts growing with rate 0.307%
    # migration rate slows between AFR-EUR
    N_EU0 = 1032
    T_EU_B = 23e3 / generation_time
    m_AF_EU = 2.5e-5
    r_EU0 = 0.00307
    N_EU1 = N_EU0 / math.exp(-r_EU0 * T_EU_B)
    
    # 5.1kya:
    # explosive growth in both AFR & EUR
    # Fu 2013  
#     T_EG = 5.1e3 / generation_time
#     r_EU = 0.0195
#     r_AF = 0.0166
#     N_EU_start = N_EU1 / math.exp(-r_EU * T_EG)
#     m_EG = 0
#     N_AF_start = N_AF / math.exp(-r_AF * T_EG)

    # Chen 2015
    T_EG = 7.26e3 / generation_time 
    r_EU = 0.0149
    r_AF = 0.00735
    N_EU_start = N_EU1 / math.exp(-r_EU * T_EG)
    m_EG = 0
    N_AF_start = N_AF / math.exp(-r_AF * T_EG)

    # Gazave 2014
#     T_EG = 3.52e3 / generation_time 
#     r_EU = 0.034
#     r_AF = 0.00735
#     N_EU = N_EU1 / math.exp(-r_EU * T_EG)
#     m_EG = 0
#     N_AF1 = N_AF / math.exp(-r_AF * T_EG)
    
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=n_afr, initial_size=N_AF_start, growth_rate=r_AF),
        msprime.PopulationConfiguration(
            sample_size=n_eur, initial_size=N_EU_start, growth_rate=r_EU)#,
    ]

    # up to 5.1kya, no migration
    migration_matrix = [
        [0, 0],
        [0, 0],
    ]
    
    demographic_events = [
        # at 5.1kya, change to slow growth rate in EUR & stop growth in AFR;
        # add migration rate
        msprime.MigrationRateChange(
            time=T_EG, rate=m_AF_EU, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EG, rate=m_AF_EU, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EG, growth_rate=r_EU0, initial_size=N_EU1, population_id=1),
        msprime.PopulationParametersChange(
            time=T_EG, growth_rate=0, population_id=0),
        
        # at 23kya, EUR growth stops and migration rates increase
        msprime.MigrationRateChange(
            time=T_EU_B, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_B, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_B, initial_size=N_EU0, growth_rate=0, population_id=1),
        
        # at 51kya, population B merges into AFR
        msprime.MassMigration(
            time=T_SPLIT, source=1, destination=0, proportion=1.0),
        msprime.PopulationParametersChange(
            time=T_SPLIT, initial_size=N_B, population_id=1),
        
        # At 148kya, instantaneous growth in AFR
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    
    if(debug):
        # Use the demography debugger to print out the demographic history
        # that we have just described.
        dd = msprime.DemographyDebugger(
            population_configurations=population_configurations,
            migration_matrix=migration_matrix,
            demographic_events=demographic_events)
        dd.print_history()
    else:
        sim = msprime.simulate(population_configurations=population_configurations,
                               migration_matrix=migration_matrix, 
                               demographic_events=demographic_events,
                               mutation_rate=mu, 
                               recombination_rate=phi, 
                               length=length,
                               random_seed=30)
    
        return sim

In [1306]:
def chen_eur_growth(mu=1.5e-8, phi=0, length=1e4, n_afr=0, n_eur=0, debug=False):
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    
    # 220kya:
    # African population constant with Ne~7300
    N_A = 7310
    
    # 148kya:
    # instantaneous growth to Ne~14000
    T_AF = 148e3 / generation_time
    N_AF = 14474
    
    N6_EU = 13143
    
    # 118kya:
    # non-AFR pops migrate OOA; bottlenecks to Ne~1800
    # migration between AFR occurs
    N_B = 1861
    T5 = 118e3 / generation_time
    T4 = T5
    m_AF_B = 15e-5
    N5_EU = 62
    N4_EU = N6_EU
    
    # 18kya:
    # 2nd EUR bottlenecks to Ne~1000 & starts growing with rate 0.307%
    # migration rate slows between AFR-EUR
    N_EU0 = 1032
    T3 = 18e3 / generation_time
    T2 = T3
#     m_AF_EU = 2.5e-5
    r_EU0 = 0 # 0.00307
#     N2_EU = 15829 # N_EU0 / math.exp(-r_EU0 * T_EU_B)
    N2_EU = 16178
    N2_AF = 26682
    N3_EU = 2020
    # 5.1kya:
    # explosive growth in both AFR & EUR
    # Fu 2013  
#     T_EG = 5.1e3 / generation_time
#     r_EU = 0.0195
#     r_AF = 0.0166
#     N_EU_start = N_EU1 / math.exp(-r_EU * T_EG)
#     m_EG = 0
#     N_AF_start = N_AF / math.exp(-r_AF * T_EG)

    # Chen 2015
#     T1_EU = 7.26e3 / generation_time 
    T1_EU = 4.95e3 / generation_time
    T1_AF = 10.01e3 / generation_time
#     r_EU = 0.0149
    r_EU = 0.022
    r_AF = 0.00735
#     N1_EU = 1.2e6 # N_EU1 / math.exp(-r_EU * T_EG)
    N1_EU = 1.261e6
    m_EG = 0
    N1_AF = 5.062e5 # N_AF / math.exp(-r_AF * T_EG)
    
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=n_afr, initial_size=N1_AF, growth_rate=r_AF),
        msprime.PopulationConfiguration(
            sample_size=n_eur, initial_size=N1_EU, growth_rate=r_EU)#,
    ]

    # up to 5.1kya, no migration
    migration_matrix = [
        [0, 0],
        [0, 0],
    ]
    
    demographic_events = [
        # at 5.1kya, change to slow growth rate in EUR & stop growth in AFR;
        # add migration rate
#         msprime.MigrationRateChange(
#             time=T_EG, rate=m_AF_EU, matrix_index=(0, 1)),
#         msprime.MigrationRateChange(
#             time=T_EG, rate=m_AF_EU, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T1_EU, growth_rate=0, initial_size=N2_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T1_AF, growth_rate=0, initial_size=N2_AF, population_id=0),
        
        # at 18kya, bottleneck + instantaneous recovery to smaller Ne
        msprime.PopulationParametersChange(
            time=T2, initial_size=N3_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T3, initial_size=N4_EU, population_id=1),
        
        # at 118kya, bottleneck + instantaneous recovery to same Ne
        msprime.PopulationParametersChange(
            time=T4, initial_size=N5_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T5, initial_size=N6_EU, population_id=1),
#         msprime.PopulationParametersChange(
#             time=T6, initial_size=N4_EU, population_id=1),
        
        
#         msprime.MigrationRateChange(
#             time=T_EU_B, rate=m_AF_B, matrix_index=(0, 1)),
#         msprime.MigrationRateChange(
#             time=T_EU_B, rate=m_AF_B, matrix_index=(1, 0)),
#         msprime.PopulationParametersChange(
#             time=T2, initial_size=N_EU0, growth_rate=0, population_id=1),
        
        # at 51kya, population B merges into AFR
#         msprime.MassMigration(
#             time=T_SPLIT, source=1, destination=0, proportion=1.0),
#         msprime.PopulationParametersChange(
#             time=T_SPLIT, initial_size=N_B, population_id=1),
        
        # At 148kya, instantaneous growth in AFR
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    
    if(debug):
        # Use the demography debugger to print out the demographic history
        # that we have just described.
        dd = msprime.DemographyDebugger(
            population_configurations=population_configurations,
            migration_matrix=migration_matrix,
            demographic_events=demographic_events)
        dd.print_history()
    else:
        sim = msprime.simulate(population_configurations=population_configurations,
                               migration_matrix=migration_matrix, 
                               demographic_events=demographic_events,
                               mutation_rate=mu, 
                               recombination_rate=phi, 
                               length=length,
                               random_seed=30)
    
        return sim

In [1274]:
chen_eur_growth(mu=mu, phi=2e-8, length=5e7, n_afr=0, n_eur=2000, debug=True)

Epoch: 0 -- 290.4 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |5.06e+05 5.99e+04        0.00735 |     0        0    
1 | 1.2e+06 1.58e+04         0.0149 |     0        0    

Events @ generation 290.4
   - Population parameter change for 1: initial_size -> 15829 growth_rate -> 0 
Epoch: 290.4 -- 400.4 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |5.99e+04 2.67e+04        0.00735 |     0        0    
1 |1.58e+04 1.58e+04              0 |     0        0    

Events @ generation 400.4
   - Population parameter change for 0: initial_size -> 26682 growth_rate -> 0 
Epoch: 400.4 -- 720.0 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |2.67e+04 2.67e+04              0 |     0        0    
1 |1.58e+04 1.58e+04              0 |     0        0    

Eve

### testing

In [1173]:
# ooa_test = out_of_africa(mu=mu, phi=2e-8, length=5e7, n_afr=0, n_eur=2000)
# # ooa_afr = out_of_africa(n_samples, mu, n_afr=n_samples, n_eur=0)
# tree_test = ooa_test.first()
# # tree_afr = ooa_afr.first()

In [1178]:
# mutation_index_test = {k: [] for k in range(2000)}
# for variant in ooa_test.variants():
#     if np.sum(variant.genotypes)==1:
#         sample = int(np.argwhere(variant.genotypes==1))
#         mutation_index_test[sample].append(variant.site.position)
# #         print(variant.site.position, variant.genotypes)

# mutation_index_test[10]

In [1172]:
# len(mutation_index_test[10])

In [1113]:
# len(ooa_test.variants())

In [786]:
# tree_sequence = msprime.simulate(
#     population_configurations=[msprime.PopulationConfiguration(
#         sample_size=10,
#         initial_size=10000, 
#         growth_rate=0.02)],
#     mutation_rate=1.5e-8, 
#     length=10e6, 
#     Ne=10000
# )

In [1174]:
# tree_test = ooa_test.first()

In [1177]:
# tree_sequence = msprime.simulate(
#     sample_size=5, Ne=1000, length=1e4, recombination_rate=2e-8)

# for tree in tree_sequence.trees():
#     print("-" * 20)
#     print("tree {}: interval = {}".format(tree.index, tree.interval))
#     print(tree.branch_length(0), tree.branch_length(1), tree.branch_length(2))
#     print(tree.draw(format="unicode"))

In [1175]:
# counter = 0
# last_branch = 0
# for tree in ooa_test.trees():
# #     print("-" * 20)
# #     print("tree {}: interval = {}".format(tree.index, tree.interval))
    
#     ext_branch_lengths = {k: [] for k in range(n_samples)}
#     for k in range(n_samples):
#         ext_branch_lengths[k]={'branch_length': tree.branch_length(k)}
                               
    
#     if ext_branch_lengths[0] != last_branch:
#         print("-" * 20)
#         print("tree {}: interval = {}".format(tree.index, tree.interval))
#         print(ext_branch_lengths[0])
#         last_branch = ext_branch_lengths[0]
    
# #     counter +=1
# #     if counter>10:
# #         break
# #     print(tree.draw(format="unicode"))

In [1132]:
# print(tree_test.draw(format="unicode"))

In [1176]:
# len(list(tree.sites()))

In [6]:
# ext_branch_lengths = []
# for i in range(8000):
#     ext_branch_lengths.append(tree.branch_length(i))

In [1151]:
# np.mean(ext_branch_lengths)

In [1184]:
del mutation_index_test

###  run with debugger to check parameters

In [8]:
n_samples = 2000 # number of haploid samples
mu = 1.5e-8 # mutation rate per generation
phi = 2e-8 # recombination rate per generation
length = 5e7 # length of chromosome in base pairs

out_of_africa(mu=mu, length=5e7, n_afr=0, n_eur=n_samples, debug=True)

Epoch: 0 -- 290.4 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |1.22e+05 1.45e+04        0.00735 |     0        0    
1 |1.32e+06 1.74e+04         0.0149 |     0        0    

Events @ generation 290.4
   - Migration rate change for (0, 1) to 2.5e-05
   - Migration rate change for (1, 0) to 2.5e-05
   - Population parameter change for 1: initial_size -> 17390.058059971307 growth_rate -> 0.00307 
   - Population parameter change for 0: growth_rate -> 0 
Epoch: 290.4 -- 920.0 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |1.45e+04 1.45e+04              0 |     0     2.5e-05 
1 |1.74e+04 2.52e+03        0.00307 |  2.5e-05     0    

Events @ generation 920.0
   - Migration rate change for (0, 1) to 0.00015
   - Migration rate change for (1, 0) to 0.00015
   - Population parameter change for 1: initial_size -> 1032 growth_rate 

### Run simulation and get tree

In [5]:
ooa_eur = chen_eur_growth(mu=1.8e-8, length=5e7, n_afr=0, n_eur=2000)
ooa_eur_rc = chen_eur_growth(mu=1.8e-8, phi=1.2e-8, length=5e7, n_afr=0, n_eur=2000)
tree_eur = ooa_eur.first()

# ooa_afr = out_of_africa(mu=mu, length=5e7, n_afr=2000, n_eur=0)
# ooa_eur_rc = out_of_africa(mu=mu, phi=2e-8, length=5e7, n_afr=2000, n_eur=0)
# tree_afr = ooa_afr.first()

NameError: name 'chen_eur_growth' is not defined

In [9]:
ooa_eur = out_of_africa(mu=mu, length=5e7, n_afr=0, n_eur=2000)
tree_eur = ooa_eur.first()

In [41]:
ooa_eur = out_of_africa(mu=1e-8, length=5e7, n_afr=0, n_eur=2000)
tree_eur = ooa_eur.first()

In [49]:
ooa_eur = out_of_africa(mu=1e-8, length=5e7, n_afr=2000, n_eur=0)
tree_eur = ooa_eur.first()

## Build index of mutation positions 

#### per haploid sample

In [50]:
mutation_index = {k: [] for k in range(n_samples)}

for site in tree_eur.sites():
    for mutation in site.mutations:
        if mutation.node<n_samples:
            mutation_index[mutation.node].append(site.position)
#             print("Mutation @ position {:.2f} over node {}".format(
#                 site.position, mutation.node))

#### per haploid sample with recombination

In [1325]:
mutation_index_rc = {k: [] for k in range(n_samples)}
for variant in ooa_eur_rc.variants():
    if np.sum(variant.genotypes)==1:
        sample = int(np.argwhere(variant.genotypes==1))
        mutation_index_rc[sample].append(variant.site.position)
#         print(variant.site.position, variant.genotypes)

# mutation_index_test[10]

In [1190]:
# len(mutation_index_rc.keys())

#### per diploid sample

##### Create random pairs of indices for when we combine 2N haploid samples into N diploid samples

In [51]:
nhap = int(n_samples/2)

r1=list(range(nhap))
r2=list(range(nhap))
pairs=r1+r2
import random
random.seed(30)
shuffle(pairs)

hap_ids = list(range(n_samples))
dip_ids = pairs
dip_index = dict(zip(hap_ids, dip_ids))

In [52]:
# Function to group haploid singletons by diploid sample

def HapToDip(mu_index, dip_index):
    nhap = len(mu_index.keys())
    ndip = int(nhap/2)
    mu_index_dip = {k: [] for k in range(ndip)}

    for k in range(nhap):
        dip_id = dip_index[k]
        mu_index_dip[dip_id].extend(mu_index[k])

    for k in range(ndip):
        mu_index_dip[k].sort()
        
    return mu_index_dip

In [53]:
mutation_index_dip = HapToDip(mutation_index, dip_index)

In [12]:
mutation_index_dip_rc = HapToDip(mutation_index_rc, dip_index)

NameError: name 'mutation_index_rc' is not defined

In [1198]:
# mutation_index_dip[0][1:4]

In [657]:
# Confirm mutations on first external branch are ordered properly
# mutation_index[0]

In [1125]:
# mutation_index_dip = {k: [] for k in range(1000)}

# for k in range(2000):
#     dip_id = dip_index[k]
#     mutation_index_dip[dip_id].extend(mutation_index_test[k])
    
# for k in range(1000):
#     mutation_index_dip[k].sort()

In [1179]:
# len(dip_ids)

In [1167]:
# len([item for sublist in mutation_index_dip.values() for item in sublist])

## Build index of inter-singleton distances

In [54]:
# get dataframe with inter-singleton distances per sample

def calcDists(index):
    dists_dict = {k: [] for k in range(len(index.keys()))}
    for k in index:
        dists_dict[k].extend([j-i for i, j in zip(index[k][:-1], index[k][1:])])
        
    dists_df = pd.concat({k: pd.Series(v) for k, v in dists_dict.items()})
    dists_df2 = pd.DataFrame(dists_df)
    dists_df2.reset_index(inplace=True)
    dists_df2.columns = ["ind_dip", "site_id", "dist"]
    return dists_df2

 #### per haploid sample

In [55]:
df2_hap = calcDists(mutation_index)

In [16]:
df2_hap.head(10)

In [19]:
df2_hap[1:10]

In [1333]:
df2_hap_rc = calcDists(mutation_index_rc)

In [1334]:
df2_hap.head(10)

,ind_dip,site_id,dist
0,0,0,1.485673e+05
1,0,1,1.363929e+06
2,0,2,7.024163e+05
3,0,3,4.428466e+05
4,0,4,4.215245e+05
5,0,5,1.932227e+05
6,0,6,3.379980e+05
7,0,7,5.945203e+05
8,0,8,2.144653e+05
9,0,9,6.174538e+05


 #### per diploid sample

In [56]:
df2_dip = calcDists(mutation_index_dip)

In [1336]:
df2_dip_rc = calcDists(mutation_index_dip_rc)

In [1337]:
df2_dip.to_csv("/mnt/norbert/home/jedidiah/projects/ERV_mutation_hotspots/data/sim_dists_dip.txt", sep="\t", index=False)
df2_dip_rc.to_csv("/mnt/norbert/home/jedidiah/projects/ERV_mutation_hotspots/data/sim_dists_dip_rc.txt", sep="\t", index=False)

In [39]:
df2_dip.size

581886

In [57]:
# df2_dip[dist<20000].size
df2_dip[df2_dip['dist']<20000].size/df2_dip.size

0.07064555420219244

In [29]:
106605/874101

0.12195959048210676

In [1128]:
# dists_hap = {k: [] for k in range(n_samples)}
# for k in mutation_index_test:
#     dists_hap[k].extend([j-i for i, j in zip(mutation_index_test[k][:-1], mutation_index_test[k][1:])])

In [1129]:
# df_hap = pd.concat({k: pd.Series(v) for k, v in dists_hap.items()})
# df2_hap = pd.DataFrame(df_hap)
# df2_hap.reset_index(inplace=True)
# df2_hap.columns = ["ind_hap", "site_id", "dist"]
# df2_hap.head(10)
# # len(df2_hap)

,ind_hap,site_id,dist
0,0,0,539899.969661
1,0,1,56593.762544
2,0,2,94610.431630
3,0,3,21046.390985
4,0,4,4645.249291
5,0,5,55777.504535
6,0,6,721375.317242
7,0,7,204891.280158
8,0,8,641582.402418
9,0,9,707945.523979


In [1130]:
# dists_dip = {k: [] for k in range(1000)}
# for k in mutation_index_dip:
#     dists_dip[k].extend([j-i for i, j in zip(mutation_index_dip[k][:-1], mutation_index_dip[k][1:])])

In [1131]:
# df_dip = pd.concat({k: pd.Series(v) for k, v in dists_dip.items()})
# df2_dip = pd.DataFrame(df_dip)
# df2_dip.reset_index(inplace=True)
# df2_dip.columns = ["ind_dip", "site_id", "dist"]
# df2_dip.head(10)

,ind_dip,site_id,dist
0,0,0,27726.581285
1,0,1,197288.318891
2,0,2,115276.617473
3,0,3,77412.157861
4,0,4,305081.065273
5,0,5,480021.442547
6,0,6,138381.153854
7,0,7,38845.840619
8,0,8,240304.270018
9,0,9,247723.595146


In [1170]:
# len(mutation_index_dip.keys())

In [1171]:
# len(df2_dip)

## Build index of external branch lengths (for models w/o recombination)

In [635]:
ext_branch_lengths = {k: [] for k in range(n_samples)}
for k in range(n_samples):
    ext_branch_lengths[k]={'branch_length': tree_eur.branch_length(k),
                          'mutation_counts': len(mutation_index[k])}
#     print(tree.branch_length(i))

In [649]:
ebl_df = pd.DataFrame.from_dict(ext_branch_lengths, orient='index')
# ebl_df = pd.DataFrame.from_dict(ext_branch_lengths)
ebl_df.reset_index(inplace=True)
ebl_df.rename(columns={'index':'ind_hap'}, inplace=True)

#### DEPRECATED: Group the 2N branches into N diploid samples, and compute the total branch lengths, weighted by the number of mutations on each of the two branches

In [651]:
ebl_df['ind_dip']=pd.Series(pairs).values
# ebl_df['wt']df.value / g.value.transform("sum") * df.wt
# ebl_df.rename(columns={'gdp':'log(gdp)'}, inplace=True)
g = ebl_df.groupby('ind_dip')
ebl_df['wt'] = ebl_df.mutation_counts / g.mutation_counts.transform("sum")
# ebl_df['wa'] = ebl_df.branch_length / g.branch_length.transform("sum") * ebl_df.wt
ebl_df['branch_length_wa'] = ebl_df.branch_length * ebl_df.wt
# ebl_df.head(10)
# ebl_df.groupby('ind_dip').wa.transform("sum").loc[ebl_df['ind_dip']==410]
ebl_df.loc[ebl_df['ind_dip']==0]



,ind_hap,branch_length,mutation_counts,ind_dip,wt,branch_length_wa
867,867,193.759860,306,0,0.857143,166.07988
1625,1625,33.951051,51,0,0.142857,4.85015


#### For diploid sample: sum the external branch lengths of the two chromosomes

In [624]:
# get weighted average of branch lengths in each diploid individual, 
# weighted by the number of mutations on each of the two branches
ebl_df_dip = pd.DataFrame(ebl_df.sort_values(by='ind_dip').groupby('ind_dip').branch_length.sum())
ebl_df_dip.reset_index(inplace=True)
ebl_df_dip.loc[ebl_df_dip['ind_dip']==410]
# ebl_df_dip.head(20)

,ind_dip,branch_length
410,410,524.063943


## Merge intervals data frame with branch lengths data frame

#### Haploid

In [678]:
df_c_hap = pd.merge(df2_hap, ebl_df, how='left', on='ind_hap')

# df_test = pd.DataFrame(df_c.head(200))
df_c_hap['cl1'] = np.where(df_c_hap['dist']<100, True, False)
df_c_hap['cl2'] = np.where((df_c_hap['dist']>100) & (df_c_hap['dist']<20000), True, False)
df_c_hap_agg = df_c_hap.groupby(['ind_hap', 'branch_length']).mean()
df_c_hap_agg.reset_index(inplace=True)
# df_c_agg = df_c_agg[df_c_agg['clustered']>0]
# df_c_agg = df_c_agg[df_c_agg['branch_length']<1900]
df_c_hap_agg.head(10)

# df_c_hap.head(10)

,ind_hap,branch_length,site_id,dist,mutation_counts,ind_dip,wt,branch_length_wa,cl1,cl2
0,0,203.659389,144.0,345381.753677,290.0,410.0,0.389262,79.276809,0.003460,0.069204
1,1,154.521566,105.0,472302.364971,212.0,406.0,0.324159,50.089560,0.000000,0.028436
2,2,90.657267,62.0,771479.200944,126.0,363.0,0.360000,32.636616,0.000000,0.016000
3,3,193.083486,146.5,338087.524097,295.0,10.0,0.551402,106.466595,0.000000,0.061224
4,4,125.187099,105.0,468835.751725,212.0,610.0,0.509615,63.797272,0.004739,0.042654
5,5,83.558921,72.5,675435.418467,147.0,220.0,0.742424,62.036169,0.000000,0.027397
6,6,197.793507,159.5,311720.450409,321.0,581.0,0.476261,94.201359,0.000000,0.068750
7,7,164.313558,122.5,401061.287385,247.0,255.0,0.600973,98.748051,0.000000,0.036585
8,8,153.269473,114.0,433097.964999,230.0,131.0,0.410714,62.949962,0.000000,0.043668
9,9,250.948283,198.0,251476.136177,398.0,451.0,0.603945,151.559054,0.000000,0.068010


#### Diploid

In [1065]:
df_c = pd.merge(df2_dip, ebl_df_dip, how='left', on='ind_dip')
# df_c[]
# df_c[df_c['ind_dip']==0]
# df_test = pd.DataFrame(df_c.head(200))
df_c['cl1'] = np.where(df_c['dist']<100, True, False)
df_c['cl2'] = np.where((df_c['dist']>100) & (df_c['dist']<20000), True, False)

aggregations = {
    'cl1': 'mean',
    'cl2': 'mean',
    'branch_length': 'max'
}

df_c_agg = df_c.groupby(['ind_dip'], as_index=False).agg(aggregations)
df_c_agg.reset_index(inplace=True)
# df_c_agg['wt'] = ebl_df.mutation_counts / g.mutation_counts.transform("sum")
# df_c_agg = df_c_agg[df_c_agg['clustered']>0]
# df_c_agg = df_c_agg[df_c_agg['branch_length']<1900]
df_c_agg.head(10)

,index,ind_dip,cl1,cl2,branch_length
0,0,0,0.000498,0.075965,227.710911
1,1,1,0.000000,0.064594,389.121197
2,2,2,0.000833,0.087464,267.113368
3,3,3,0.000864,0.089378,314.458521
4,4,4,0.000000,0.059438,175.671812
5,5,5,0.000370,0.101962,144.849699
6,6,6,0.000000,0.047909,507.452973
7,7,7,0.000829,0.089101,338.914767
8,8,8,0.000579,0.066319,607.445186
9,9,9,0.000606,0.092690,362.617259


### Plot CDF of inter-singleton distances (diploid data)

#### Matplotlib

In [1210]:
# fig, ax = plt.subplots(facecolor='white')

# x = df_c.loc[df_c['ind_dip']<10]['dist'].values
# sim_dists = df_c['dist'].values
sim_dists = df2_dip['dist'].values
sim_dists.sort()

sim_dists_rc = df2_dip_rc['dist'].values
sim_dists_rc.sort()


num_bins = 50000
# b10

In [1229]:
b10bins = np.multiply(x.reshape(7,1), np.arange(1,11)).flatten()

sim_counts, sim_bin_edges = np.histogram(sim_dists, bins=b10bins, normed=True)
sim_counts_rc, sim_bin_edges_rc = np.histogram(sim_dists_rc, bins=b10bins, normed=True)
# sim_counts = sim_counts[1::5]
# sim_bin_edges = sim_bin_edges[1::5]
sim_cdf = np.cumsum(sim_counts)
sim_cdf_rc = np.cumsum(sim_counts_rc)
# plt.plot(sim_bin_edges[1:], sim_cdf/sim_cdf[-1])

In [1212]:
b10bins

array([       1,        2,        3,        4,        5,        6,
              7,        8,        9,       10,       10,       20,
             30,       40,       50,       60,       70,       80,
             90,      100,      100,      200,      300,      400,
            500,      600,      700,      800,      900,     1000,
           1000,     2000,     3000,     4000,     5000,     6000,
           7000,     8000,     9000,    10000,    10000,    20000,
          30000,    40000,    50000,    60000,    70000,    80000,
          90000,   100000,   100000,   200000,   300000,   400000,
         500000,   600000,   700000,   800000,   900000,  1000000,
        1000000,  2000000,  3000000,  4000000,  5000000,  6000000,
        7000000,  8000000,  9000000, 10000000])

In [1225]:
from scipy import stats
stats.describe(sim_dists)

DescribeResult(nobs=218514, minmax=(1.7345882952213287, 10158576.187677681), mean=226347.65996071434, variance=69114726263.36687, skewness=3.7975456177029674, kurtosis=43.43822438009587)

In [1228]:
stats.describe(sim_dists_rc)

DescribeResult(nobs=219724, minmax=(0.12388594914227724, 8052620.538741935), mean=225232.8606784507, variance=61399461042.96518, skewness=2.6164602630974487, kurtosis=15.186805114266711)

In [1233]:
# exp_sites = np.random.exponential(230000,400)
# exp_sites.sort()

exp_dists = np.random.exponential(226347,218514)
exp_dists.sort()
# exp_sites

In [1234]:
# exp_dists = []
# exp_dists.extend([j-i for i, j in zip(exp_sites[:-1], exp_sites[1:])])
# exp_dists.sort()

exp_counts, exp_bin_edges = np.histogram(exp_dists, bins=b10bins, normed=True)
# exp_counts = exp_counts[1::5]
# exp_bin_edges = exp_bin_edges[1::5]
exp_cdf = np.cumsum(exp_counts)

#### Plotly

In [1241]:
# cumsum = np.cumsum(x)
layout = go.Layout(
    xaxis=dict(
        title='Distance',
        range=[1,6],
        titlefont=dict(
#             family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        type="log"
    ),
    
    yaxis=dict(
#         autorange=True
#         type="log"
#         range=[0,0.4]
    )
)

tr1 = go.Scatter(
    x = exp_bin_edges[1:], 
    y = exp_cdf/exp_cdf[-1],
    name = "naive exponential",
    marker = dict(color = 'rgb(10, 25, 120)')
)

tr2 = go.Scatter(
    x = sim_bin_edges[1:], 
    y = sim_cdf/sim_cdf[-1],
    name = "simulated",
    marker = dict(color = 'rgb(150, 25, 120)')
)

tr3 = go.Scatter(
    x = sim_bin_edges_rc[1:], 
    y = sim_cdf_rc/sim_cdf_rc[-1],
    name = "simulated with recombination",
    marker = dict(color = 'rgb(50, 25, 20)')
)

# tr1 = go.Histogram(x=x, histnorm='probability')
data = [tr1, tr2, tr3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='dist_cdf')

### Plot branch length vs # singletons <20kb apart

In [1066]:
layout = go.Layout(
    xaxis=dict(
        title='External Branch Length (generations)',
        titlefont=dict(
#             family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )#,
#         type="log"
    ),
    yaxis=dict(
        title='Fraction of intervals <20kb',
        titlefont=dict(
#             family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )#,
#         type="log"
    )
)

#### Haploid

In [1057]:
# Create a trace
trace = go.Scatter(
    x = df_c_hap_agg.branch_length,
    y = df_c_hap_agg.cl2,
    mode = 'markers',
#     text= df_c_hap_agg.site_id,
    marker= dict(
        size= 6,
        line= dict(width=1),
        opacity= 0.6
    )
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='ext_branch_length_vs_n_hap')

#### Diploid

In [1067]:
# Create a trace
trace = go.Scatter(
    x = df_c_agg.branch_length,
    y = df_c_agg.cl2,
    mode = 'markers',
#     text= df_c_agg.site_id,
    marker= dict(
        size= 6,
        line= dict(width=1),
        opacity= 0.6
    )
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='ext_branch_length_vs_n')

In [1035]:
# df_c_agg.head(20)

,ind_dip,branch_length,site_id,dist,cl1,cl2
0,0,227.710911,4014,2.442391e+06,True,True
1,1,389.121197,3312,3.005274e+06,False,True
2,2,267.113368,4801,1.619195e+06,True,True
3,3,314.458521,4631,2.099762e+06,True,True
4,4,175.671812,3061,3.285773e+06,False,True
5,5,144.849699,5403,1.444376e+06,True,True
6,6,507.452973,2295,3.841548e+06,False,True
7,7,338.914767,4825,1.583423e+06,True,True
8,8,607.445186,3452,2.529253e+06,True,True
9,9,362.617259,4951,1.481148e+06,True,True


#### Get correlation between % clustered and branch length for haploid sample

In [1058]:
# df_c_agg.corr(method='spearman')
df_c_agg.corr(method='spearman')

,index,ind_dip,branch_length,cl1,cl2,dist
index,1.000000,1.000000,-0.027779,0.025445,0.027071,-0.024946
ind_dip,1.000000,1.000000,-0.027779,0.025445,0.027071,-0.024946
branch_length,-0.027779,-0.027779,1.000000,-0.000457,0.025378,0.007188
cl1,0.025445,0.025445,-0.000457,1.000000,0.387924,-0.374806
cl2,0.027071,0.027071,0.025378,0.387924,1.000000,-0.883627
dist,-0.024946,-0.024946,0.007188,-0.374806,-0.883627,1.000000


### Calculate total proportion of inter-singleton distances that occur below MNM threshold

In [313]:
len(list(tree.sites()))

795907

In [1068]:
# & (df_c['branch_length']<20)
cluster_fps = len(df_c[(df_c['cl1']==True)])
tot_singletons = len(df_c)
cluster_fps_prop = cluster_fps/tot_singletons
cluster_fps_prop

0.0005257486595221328

In [542]:
tot_singletons

432597

In [543]:
tot_singletons/795907

0.5435270703738